In [1]:
import graphlab

In [4]:
apps_data_sf = graphlab.SFrame.read_csv("server/src/data/i5tptf5jqiq7oa1toeqo0ekgf3_apps.csv", verbose=False)

In [5]:
graphlab.canvas.set_target('ipynb')

apps_data_sf.show()

In [6]:
import graphlab.aggregate as agg
grouped_words_sf = apps_data_sf.groupby('session_id', {
    'words': agg.CONCAT('word'),
    'timestamp_var': agg.VAR('timestamp'), 
    'timestamp_std': agg.STD('timestamp'),
    'app': agg.SELECT_ONE('app')})  

In [9]:
def get_normalized_tf_idf(sf):
    n = len(sf['words'])
    tf_idf_accum = {}

    for key, value in sf['tf_idf'].iteritems():
        tf_idf_accum[key] = value / n

    return tf_idf_accum


In [10]:
grouped_words_sf['bow'] = graphlab.text_analytics.count_words(grouped_words_sf['words'])
grouped_words_sf['tf_idf'] = graphlab.text_analytics.tf_idf(grouped_words_sf['bow'])
grouped_words_sf['normalized_tf_idf'] = grouped_words_sf.apply(get_normalized_tf_idf)

# With classifier approach

In [38]:
total = len(grouped_words_sf)

feature_sets = [['normalized_tf_idf', 'timestamp_var', 'timestamp_std'],
                ['normalized_tf_idf'],
                ['tf_idf', 'timestamp_var', 'timestamp_std'],
                ['tf_idf']]


for features in feature_sets:
    n_right = 0
    for i in range(len(grouped_words_sf)):
        e = grouped_words_sf.head(i+1).tail(1)
        to_use_grouped_words = grouped_words_sf[:i].append(grouped_words_sf[i+1:])
        m = graphlab.logistic_classifier.create(
                to_use_grouped_words, target='app', verbose=False, features=features)
        if e[0]['app'] == m.predict(e)[0]: #, k=1, verbose=False)['reference_label'][0]:
            n_right += 1

    print features
    print n_right / float(total) * 100
    print n_right, total
    

['normalized_tf_idf', 'timestamp_var', 'timestamp_std']
70.0
21 30
['normalized_tf_idf']
70.0
21 30
['tf_idf', 'timestamp_var', 'timestamp_std']
80.0
24 30
['tf_idf']
83.3333333333
25 30


# With nearest neightbor approach

In [35]:
total = len(grouped_words_sf)

feature_sets = [['normalized_tf_idf', 'timestamp_var', 'timestamp_std'],
                ['normalized_tf_idf'],
                ['tf_idf', 'timestamp_var', 'timestamp_std'],
                ['tf_idf']]


for features in feature_sets:
    n_right = 0
    for i in range(len(grouped_words_sf)):
        e = grouped_words_sf.head(i+1).tail(1)
        to_use_grouped_words = grouped_words_sf[:i].append(grouped_words_sf[i+1:])
        m = graphlab.nearest_neighbors.create(
                to_use_grouped_words, label='app', distance='euclidean', method='brute_force', features=features,
                verbose=False)
        if e[0]['app'] == m.query(e, k=1, verbose=False)['reference_label'][0]:
            n_right += 1

    print features
    print n_right / float(total) * 100
    print n_right, total


['normalized_tf_idf', 'timestamp_var', 'timestamp_std']
36.6666666667
11 30
['normalized_tf_idf']
80.0
24 30
['tf_idf', 'timestamp_var', 'timestamp_std']
36.6666666667
11 30
['tf_idf']
66.6666666667
20 30


## Best seems to be classifier with only tf_idf, will try again with double the data